In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import tensorflow as tf
import pandas as pd
import numpy as np
import cv2

In [3]:
# padding='same' : 텐서플로우가 자동으로 패딩을 삽입해 입력값과 출력값의 크기를 맞춰줌
# padding='valid' : 텐서플로우가 자동으로 패딩을 적용하지 않고 필터를 적용해서 출력값의 크기가 작이짐
 
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            layers.Input(input_shape),
            
            # 1st
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),    # 64 : 필터 수(출력), 3 : 3x3 필터 크기 => 각 필터의 가중치는 다름
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),    # 배치별 특징 값을 정규화(relu로 인한 다양한 값 생성(local optimal)) => 학습 속도 증가
            layers.Dropout(0.5),
            
            # 2nd
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),    # 128 : 필터 수(출력), 3 : 3x3 필터 크기 => 각 필터의 가중치는 다름
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),    # 배치별 특징 값을 정규화(relu로 인한 다양한 값 생성(local optimal)) => 학습 속도 증가
            layers.Dropout(0.3),
            
            # FC
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),    # 2nd에서 출력이 128개
            layers.Dense(1, activation='sigmoid')    # 마지막 결과 : 맞는지 여부(2진 논리)
        ]
    )
    
    return model

In [4]:
input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)

In [5]:
import math

class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, fold, image_size, mode='train',  shuffle=True):
        self.batch_size = batch_size
        self.fold = fold
        self.image_size = image_size
        self.mode = mode
        self.shuffle = shuffle
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
            
        self.on_epoch_end()
        
    # sample() : 전체 row에서 몇 %의 데이터를 return 할 것인지 설정
    # frac : 데이터프레임으로 부터 특정 비율로 무작위 표본 추출을 하고 싶으면 0~1 사이의 부동소수점 입력
    # reset_index() : drop=True 옵션을 설정하면 인덱스 열을 보통의 자료열로 설정하는 것이 아니라 버림
    # 여기서 인데스 열이란 데이터프레임 맨 앞 열에 생성되는 인덱스 열을 말함
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
            
    # len() 함수를 재구현  (공유파일 첫 달에서 스페셜매소드 파일에서 다뤘음)
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
    
    # 배열을 쓸 때 자동으로 불러지는 것
    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = (idx + 1) * self.batch_size
        data = self.df.iloc[start:end]
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        # iterrows
        # 데이터프레임에서 row의 개수만큼 반복
        # 튜플로 리턴[index, 행정보(Series)]
        for _, r in data.iterrows():             
            file_name = r['file_name']
            image = cv2.imread(f'D:/data_ai/5_AI/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.image_size, self.image_size))
            image = image / 255    # 정규화
            
            label = int(r['species']) - 1    # 밑에서 class_name 인덱스와 맞추기위해 (-1)해줌
            
            batch_x.append(image)
            batch_y.append(label)
            
        return batch_x, batch_y

In [6]:
csv_path = 'D:/data_ai/5_AI/kfolds.csv'

train_generator = DataGenerator(
    # 옵션
    batch_size = 128, 
    csv_path = csv_path, 
    fold = 1, 
    image_size = 256, 
    mode = 'train', 
    shuffle = True
)

valid_generator = DataGenerator(
    # 옵션
    batch_size = 128, 
    csv_path = csv_path, 
    fold = 1, 
    image_size = 256, 
    mode = 'val', 
    shuffle = True
)

In [7]:
# early_stopping : Epoch을 많이 돌린 후 특정 시점에 멈추게 함
# monitor : EarlyStopping의 기준이 되는 값. 예) val_loss 더 이상 감소되지 않을 경우 EarlyStopping 적용
# patience : Trainning이 진행됨에도 더 이상 monitor되는 값의 개선이 없을 경우 몇 번 epoch을 진행할 지 설정
# mode : monitor되는 값이 최소가 돼야 하는지, 최대가 돼야 하는지 설정
# restore_best_weights : true라면 trainning이 끝난 후, model의 weight를 monitor하고 있던 값이 가장 좋았을 경우의 weight로 복원,
# false라면 마지막 trainning이 끝난 후의 weight로 설정

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, mode='min', restore_best_weights=False
    
)

In [8]:
# reduce_on_plateau : 모델의 개선이 없을 경우 learning rate를 조절해 모델의 개선을 유도함
# monitor : reduce_on_plateau의 기준이 되는 값. 예) val_loss 더 이상 감소되지 않을 경우 reduce_on_plateau 적용
# factor : learning rate를 얼마나 감쇠킬지 정하는 값. 새로운 learning rate는 기존 learning rate * factor
# patience : trainning이 진해됨에도 더 이상 monitor되는 값의 개선이 없을 경우 최적의 monitor 값을 기준으로
# 몇 번의 epoch을 진행하고 learning rate를 조절할지의 값을 설정
# verbose : 세세한 출력 방법. learning rate하는 것이 콘솔에 출력

reduce_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min', min_lr=0.0001
)

In [9]:
# model_checkpoint : 모델의 경로 설정
# 모델 경로를 '{epoch:02d}-{val_losee:.2f}.hdf5'라고 하면 앞의 명시한 문자열로 파일을 저장
# 예) 01-0.12.h5
# save_weights_only : True(weight만 저장), False(모델, layer, weight 모두 저장)
# save_best_only : True(모델의 정확도가 최고값을 갱신했을 때만 저장), False(매회 저장)

filepath = '{epoch:02d}-{val_losee:.2f}.hdf5'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min'
)

In [1]:
history = model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs=10,
    callbacks=[
        early_stopping,
        reduce_on_plateau,
        model_checkpoint
    ]
)

# 만족할 만한 결과 안나옴(60~70%)
# sequential이 cnn에 적합하지 않음
# 보족한 자료 + 데이터 추가 못함 => 기존의 자료를 뿔려야함

NameError: name 'model' is not defined